**Reading data from csv**

In [ ]:
# 1.Data Cleaning and Preparation for Customers.csv
import pandas as pd
#Load the customer.csv into dataframe
customer_df=pd.read_csv(r'C:\Users\USER\Guvi python\Dataspark\Customers.csv',encoding='unicode_escape')
#Check is null values ae present
print(customer_df.isnull().sum()) 
# Drop column as it the duplicate of State column
customer_df.drop(['State Code'],axis=1,inplace=True) 
customer_df.dropna(inplace=True)
# to check the datatypes
print(customer_df.info())
#Change the datatype for the date column
customer_df['Birthday']=pd.to_datetime(customer_df['Birthday'],errors='coerce')
#print(customer_df.isnull().sum())
#Write the cleaned data into '.csv' file
customer_df.to_csv('CleanedCustomer.csv')
customer_df

In [ ]:
# 2.Data Cleaning and Preparation for Exchange_Rates.csv
#Load the Exchange_Rates.csv file into the dataframe
exchange_rates_df=pd.read_csv(r'C:\Users\USER\Guvi python\Dataspark\Exchange_Rates.csv',encoding='unicode_escape')
# To check missing values
print(exchange_rates_df.isnull().sum())
# To check the datatypes
print(exchange_rates_df.info())
#Changing the datatype for date column
exchange_rates_df['Date']=pd.to_datetime(exchange_rates_df['Date'],errors='coerce')
#Renaming the Currency column to match the Sales.csv
exchange_rates_df.rename(columns={'Currency': 'Currency Code'}, inplace=True)
#print(exchange_rates_df.info())
#Write the cleaned data into '.csv' file
exchange_rates_df.to_csv('CleanedExchangeRates.csv')
exchange_rates_df

In [ ]:
# 3.Data Cleaning and Preparation for Products.csv
#Load the Products.csv into dataframe
products_df=pd.read_csv(r'C:\Users\USER\Guvi python\Dataspark\Products.csv',encoding='unicode_escape')
print(products_df.isnull().sum())# To check missing values
print(products_df.info()) # to check the datatypes
#Removing the special characters and coverting to numeric
products_df['Unit Cost USD'] = products_df['Unit Cost USD'].str.replace("$","")
products_df['Unit Cost USD'] = products_df['Unit Cost USD'].str.replace(",","")
products_df['Unit Cost USD'] = pd.to_numeric(products_df['Unit Cost USD'])

products_df['Unit Price USD'] = products_df['Unit Price USD'].str.replace("$","")
products_df['Unit Price USD'] = products_df['Unit Price USD'].str.replace(",","")
products_df['Unit Price USD'] = pd.to_numeric(products_df['Unit Price USD'])
#Write the cleaned data into '.csv' file
products_df.to_csv('CleanedProducts.csv')
products_df

In [ ]:
# 4.Data Cleaning and Preparation for Sales.csv
#Load the data from Sales.csv in to the dataframe
sales_df=pd.read_csv(r'C:\Users\USER\Guvi python\Dataspark\Sales.csv',encoding='unicode_escape')
print(sales_df.isnull().sum())# To check missing values
print(sales_df.info()) # To check the datatypes
#Changing the datatype for date columns
sales_df['Order Date']=pd.to_datetime(sales_df['Order Date'],errors='coerce')
sales_df['Delivery Date']=pd.to_datetime(sales_df['Delivery Date'],errors='coerce')
# Interpolate to fill missing dates in the DataFrame
sales_df['Delivery Date'] = sales_df['Delivery Date'].interpolate(method='pad')
# Backfill the 'Delivery Date' column since the first value is null
sales_df['Delivery Date'] = sales_df['Delivery Date'].fillna(method='bfill')
#Write the cleaned data into '.csv' file
sales_df.to_csv('CleanedSales.csv')
sales_df

In [ ]:
# 5.Data Cleaning and Preparation for Stores.csv
#Load the data from Stores.csv in to the dataframe
stores_df=pd.read_csv(r'C:\Users\USER\Guvi python\Dataspark\Stores.csv',encoding='unicode_escape')
print(stores_df.isnull().sum())# To check missing values
print(stores_df.info()) # To check the datatypes
#print(stores_df[stores_df.isnull().any(axis=1)])
stores_df['Square Meters'].fillna(0,inplace=True) # Assigned 0 for Online store

#Change the datatype for int and date columns
stores_df['Square Meters']=stores_df['Square Meters'].astype('int64')
stores_df['Open Date']=pd.to_datetime(stores_df['Open Date'],errors='coerce')
#Write the cleaned data into '.csv' file
stores_df.to_csv('CleanedStores.csv')
stores_df

**Creating SQL connection and loading data**

In [ ]:
#Install mysql connector
!pip install mysql-connector-python

In [ ]:
#import and establish connection
import mysql.connector
#add in your db credentials
con= mysql.connector.connect(
        host='localhost',
        user='root',
        password='12345678'
    , database='capstone'   #needed to connect to database directly once the database is created
) 
cursor=con.cursor(buffered=True)
cursor

In [ ]:
#create a database
query='create database Capstone'
cursor.execute(query)

In [ ]:
#use the created database
query='use capstone'
cursor.execute(query)

In [ ]:
#To check whether the correct database is selected
query='select database()'
cursor.execute(query)
for db in cursor:
    print(db)

**Create table**

In [ ]:
#1.Create a table Customers
query='''create table Customers(CustomerKey INT PRIMARY KEY, Gender VARCHAR(10),Name VARCHAR(255),City VARCHAR(255),State VARCHAR(255),
                                    Zip_Code VARCHAR(20),Country VARCHAR(255),Continent VARCHAR(255),Birthday DATETIME)'''
cursor.execute(query)

In [ ]:
#To check the table structure for Customers
query="describe Customers"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#2.Create a table Exchange_rates
query='''create table Exchange_rates(Date DATETIME,Currency_code VARCHAR(10),Exchange DECIMAL(10,4))'''
cursor.execute(query)
#To check the table structure for Exchange_rates
query="describe Exchange_rates"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#3.Create a table Products
query='''create table Products(ProductKey INT PRIMARY KEY, Product_Name VARCHAR(255), Brand VARCHAR(50), Color VARCHAR(50),
                                Unit_Cost_USD DECIMAL(20,2),Unit_Price_USD DECIMAL(20,2), SubcategoryKey INT, 
                                Subcategory VARCHAR(50), CategoryKey INT,Category VARCHAR(50))'''
cursor.execute(query)
#To check the table structure for Products
query="describe Products"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#4. Create a Sales table
query='''create table Sales(Order_Number INT NOT NULL,Line_Item INT NOT NULL, Order_Date DATETIME, Delivery_Date DATETIME,
                            CustomerKey INT, StoreKey INT, ProductKey INT, Quantity INT, Currency_code VARCHAR(10),
                               CONSTRAINT Sales_UC UNIQUE (Order_Number,Line_Item),
                               FOREIGN KEY (CustomerKey) REFERENCES Customers(CustomerKey),
                               FOREIGN KEY (StoreKey) REFERENCES Stores(StoreKey),
                               FOREIGN KEY (ProductKey) REFERENCES Products(ProductKey)
                               )'''
cursor.execute(query)
#To check the table structure for Stores
query="describe Sales"
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#5.Create a table Stores
query='''create table Stores(StoreKey INT PRIMARY KEY, Country VARCHAR(50), State VARCHAR(50), Square_Meters INT, Open_Date DATETIME)'''
cursor.execute(query)
#To check the table structure for Stores
query="describe Stores"
cursor.execute(query)
for data in cursor:
    print(data)

**Loading Data**

In [ ]:
#1.Insert data into Customers table

query='''insert into Customers(CustomerKey,Gender,Name,City,State,Zip_Code,Country,Continent,Birthday) 
                                values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
# Insert data into the table
for index, row in customer_df.iterrows():
    values = (row['CustomerKey'], row['Gender'], row['Name'],row['City'], row['State'], 
              row['Zip Code'],row['Country'], row['Continent'], row['Birthday'])
    cursor.execute(query, values)

# Commit changes and close the connection
con.commit() 

In [ ]:
#2.Insert data into Exchange_rates table

query='''insert into Exchange_rates(Date,Currency_code,Exchange) values(%s,%s,%s)'''
# Insert data into the table
for index, row in exchange_rates_df.iterrows():
    values = (row['Date'], row['Currency code'], row['Exchange'])
    cursor.execute(query, values)

# Commit changes and close the connection
con.commit() 

In [ ]:
#3.Insert data into Products table

query='''insert into Products(ProductKey, Product_Name, Brand, Color, Unit_Cost_USD, Unit_Price_USD, SubcategoryKey, Subcategory, 
                                    CategoryKey, Category) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
# Insert data into the table
for index, row in products_df.iterrows():
    values = (row['ProductKey'], row['Product Name'], row['Brand'],row['Color'], row['Unit Cost USD'], row['Unit Price USD'],
              row['SubcategoryKey'], row['Subcategory'], row['CategoryKey'],row['Category'])
    cursor.execute(query, values)

# Commit changes and close the connection
con.commit() 

In [ ]:
#4.Insert data into Sales table

query='''insert into Sales(Order_Number, Line_Item, Order_Date, Delivery_Date,CustomerKey, StoreKey, ProductKey, Quantity, Currency_Code) 
                            values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
# Insert data into the table
for index, row in sales_df.iterrows():
    values = (row['Order Number'], row['Line Item'], row['Order Date'],row['Delivery Date'], row['CustomerKey'], row['StoreKey'],
              row['ProductKey'], row['Quantity'], row['Currency Code'])
    cursor.execute(query, values)

# Commit changes and close the connection
con.commit() 

In [ ]:
#5.Insert data into Stores table

query='''insert into Stores(StoreKey, Country, State, Square_Meters, Open_Date) values(%s,%s,%s,%s,%s)'''
# Insert data into the table
for index, row in stores_df.iterrows():
    values = (row['StoreKey'], row['Country'], row['State'],row['Square Meters'], row['Open Date'])
    cursor.execute(query, values)

# Commit changes and close the connection
con.commit() 

**Viewing the table contents**

In [ ]:
#1.View Customers table
query='select * from Customers'
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#2.View Exchange_rates table
query='select * from Exchange_rates'
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#3.View Products table
query='select * from Products'
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#4.View Sales table
query='select count(*) from Sales'
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#5.View Stores table
query='select * from Stores'
cursor.execute(query)
for data in cursor:
    print(data)

In [ ]:
#Merging datasets where necessary for analysis
import pandas as pd
merged_df = pd.merge(sales_df, exchange_rates_df, how='inner', left_on=['Order Date', 'Currency Code'], right_on=['Date', 'Currency Code'])
merged_df = merged_df.merge(stores_df, on='StoreKey').merge(products_df, on='ProductKey').merge(customer_df, on='CustomerKey')
merged_df.to_csv('salesinfo.csv')

In [ ]:
merged_df

In [ ]:
# !pip install plotly
import matplotlib.pyplot as plt
# Extract year
merged_df['Year'] = merged_df['Order Date'].dt.year

# Calculate total sales (Quantity * Unit Price USD)
merged_df['Total Sales'] = merged_df['Quantity'] * merged_df['Unit Price USD']

# Group by year and sum total sales
yearly_sum = merged_df.groupby('Year')['Total Sales'].sum().reset_index()

# Line plot to see Year vs Yearly Sales details 
plt.plot(yearly_sum['Year'], yearly_sum['Total Sales'], color='blue', linestyle='solid', marker='*')
plt.grid(True)
plt.xlabel("Year")
plt.ylabel("Total Sales (USD)")
plt.title("Total Sales by Year")
plt.xticks(yearly_sum['Year'])  # Ensure all years are labeled
plt.tight_layout()
plt.show()